<a href="https://colab.research.google.com/github/poojagolande256-ui/Learning_Studio/blob/main/multiclass_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1 Change Runtime type-> T4 GPU

Step 2 Data Ig

In [ ]:
S